In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from keras.applications.imagenet_utils import decode_predictions

In [ ]:
from keras.preprocessing import image as keras_image

def transform_img_fn(path_list):
    out = []
    for img_path in path_list:
        img = keras_image.load_img(img_path, target_size=(224, 224))
        x = keras_image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = inc_net.preprocess_input(x)
        out.append(x)
    return np.vstack(out)

In [ ]:
import foolbox
import keras
import numpy as np
from keras.applications import inception_v3 as inc_net

# instantiate model
keras.backend.set_learning_phase(0)
kmodel = inc_net.InceptionV3()

In [ ]:
fmodel = foolbox.models.KerasModel(kmodel, bounds=(-1, 1))

In [ ]:
image = transform_img_fn(["./strawberry.jpg"])[0]
label = 949

In [ ]:
%%time
# apply attack on source image
attack = foolbox.attacks.FGSM(fmodel)
adversarial = attack(image[:,:,::-1], label)

In [ ]:
# I'm dividing by 2 and adding 0.5 because of how this Inception represents images
plt.imshow(image / 2 + 0.5)
preds = kmodel.predict(np.expand_dims(image, axis=0))
for x in decode_predictions(preds):
    print(x)

In [ ]:
# I'm dividing by 2 and adding 0.5 because of how this Inception represents images
plt.imshow(adversarial / 2 + 0.5)
preds = kmodel.predict(np.expand_dims(adversarial, axis=0))
for x in decode_predictions(preds):
    print(x)

## Lime

In [ ]:
%load_ext autoreload
%autoreload 2
import os,sys
try:
    import lime
except:
    sys.path.append(os.path.join('..', '..')) # add the current directory
    import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries

In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
%%time
explanation_adv = explainer.explain_instance(adversarial, kmodel.predict, top_labels=5, hide_color=0, num_samples=200)

In [ ]:
label_img = 960
temp, mask = explanation_adv.get_image_and_mask(label_img, positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))